In [17]:
import numpy as np
import pandas as pd
import warnings
import re
warnings.filterwarnings("ignore")

In [18]:
df = pd.read_excel("../data/Dailylog data -lifelog/Lifestyle - Step count.xlsx")
df.columns=df.columns.str.lower()

In [19]:
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d %H:%M:%S')

In [20]:
df.rename(columns={"non-identifying keys":"id"},inplace=True)

In [21]:
steps = df["measure (-1: no value)"].apply(lambda x: list(map(int, re.findall(r'\d', str(x)))))
steps = steps.apply(lambda x: x[:1440] if len(x) > 1440 else x + [0] * (1440 - len(x)))

steps_array = np.array(steps.tolist())
hourly_steps = steps_array.reshape(-1, 24, 60)

hourly_variances = np.var(hourly_steps, axis=2)
hourly_max = np.max(hourly_steps, axis=2)
hourly_mean = np.mean(hourly_steps, axis=2)

daily_sum = np.sum(hourly_variances, axis=1)
daily_variance = np.mean(hourly_variances, axis=1)
daily_max = np.max(hourly_max, axis=1)
daily_mean = np.mean(hourly_mean, axis=1)

df['daily_step_variance'] = daily_variance
df['daily_max_steps'] = daily_max
df['daily_mean_steps'] = daily_mean
df['daly_sum_steps']=daily_sum


print(df[['date', 'daily_step_variance', 'daily_max_steps', 'daily_mean_steps']].dropna())

           date  daily_step_variance  daily_max_steps  daily_mean_steps
0    2021-08-10                  0.0                0               0.0
1    2021-08-09                  0.0                0               0.0
2    2021-08-08                  0.0                0               0.0
3    2021-08-07                  0.0                0               0.0
4    2021-08-06                  0.0                0               0.0
...         ...                  ...              ...               ...
8527 2023-09-17                  0.0                0               0.0
8528 2023-09-16                  0.0                0               0.0
8529 2023-09-15                  0.0                0               0.0
8530 2023-09-14                  0.0                0               0.0
8531 2023-09-13                  0.0                0               0.0

[8532 rows x 4 columns]


In [22]:
df.drop(columns=["measurement types","units of measure","measure (-1: no value)","total steps"],inplace=True)

In [23]:
df.set_index(["id","date"],inplace=True)
df.to_csv("../data_processed/daily_lifelog/step_count.csv")